In [27]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

# ============================================================
# 0. 路径配置
# ============================================================
BASE_DIR = Path("./data")
ATH_PATH = BASE_DIR / "summerOly_athletes.csv"
MEDAL_PATH = BASE_DIR / "summerOly_medal_counts.csv"
HOST_PATH = BASE_DIR / "summerOly_hosts.csv"
PROGRAM_PATH = BASE_DIR / "summerOly_programs.csv"
OUT_TRAIN = BASE_DIR / "train_table_v6_final.csv"


# ============================================================
# 1. 数据加载与清洗函数
# ============================================================
def load_and_clean_data():
    """加载原始数据并进行基础清洗"""
    print(">>> [1/4] 正在读取原始数据...")
    
    # 读取数据
    athletes_df = pd.read_csv(ATH_PATH, encoding="utf-8")
    medals_df = pd.read_csv(MEDAL_PATH, encoding="utf-8")
    hosts_df = pd.read_csv(HOST_PATH, encoding="utf-8")
    programs_df = pd.read_csv(PROGRAM_PATH, encoding="latin1")
    
    # 格式化年份
    athletes_df = athletes_df[athletes_df['Year'] != 1906]  # 排除1906届
    athletes_df["Year"] = athletes_df["Year"].astype(int)
    medals_df["Year"] = medals_df["Year"].astype(int)
    hosts_df["Year"] = hosts_df["Year"].astype(int)
    
    return athletes_df, medals_df, hosts_df, programs_df


def clean_noc_and_mapping(athletes_df, medals_df):
    """清洗NOC代码并构建映射关系"""
    print(">>> [2/4] 清洗 NOC 与构建映射...")
    
    # 清洗无效NOC
    invalid_nocs = ['AIN', 'EOR', 'ROT', 'IOA', 'IOP', 'ZZX', 'MIX', 'UNK']
    athletes_cleaned = athletes_df[~athletes_df['NOC'].isin(invalid_nocs)].copy()
    athletes_cleaned.loc[athletes_cleaned['NOC'] == 'ROC', 'NOC'] = 'RUS'
    
    # 过滤体育项目
    if 'Sport' in athletes_cleaned.columns:
        athletes_cleaned = athletes_cleaned[athletes_cleaned['Sport'] != 'Art Competitions']
    if 'Season' in athletes_cleaned.columns:
        athletes_cleaned = athletes_cleaned[athletes_cleaned['Season'] == 'Summer']
    
    # 建立NOC映射
    name_to_code_map = athletes_cleaned.set_index('Team')['NOC'].to_dict()
    manual_patches = {
        "Soviet Union": "URS", "East Germany": "GDR", "West Germany": "FRG",
        "Great Britain": "GBR", "United States": "USA", "China": "CHN",
        "People's Republic of China": "CHN", "ROC": "RUS", "Russia": "RUS",
        "Unified Team": "EUN", "Korea, South": "KOR", "South Korea": "KOR"
    }
    name_to_code_map.update(manual_patches)
    
    # 清洗奖牌数据
    medals_cleaned = medals_df.copy()
    medals_cleaned['NOC_Code'] = medals_cleaned['NOC'].map(name_to_code_map).fillna(medals_cleaned['NOC'])
    medals_cleaned.loc[medals_cleaned['NOC_Code'] == 'ROC', 'NOC_Code'] = 'RUS'
    medals_cleaned = medals_cleaned[~medals_cleaned['NOC_Code'].isin(invalid_nocs)]
    
    return athletes_cleaned, medals_cleaned, name_to_code_map


# ============================================================
# 执行数据加载与清洗
# ============================================================
ath, med, hst, prog = load_and_clean_data()
ath, med, name_to_code_map = clean_noc_and_mapping(ath, med)


>>> [1/4] 正在读取原始数据...
>>> [2/4] 清洗 NOC 与构建映射...


In [28]:
# ============================================================
# 2. 构建基础特征函数
# ============================================================
def build_basic_features(athletes_df, medals_df, programs_df):
    """构建宏观和微观基础特征"""
    print(">>> [3/4] 构建特征工程 (宏观+微观)...")
    
    # --- A. 宏观特征 ---
    year_cols = [c for c in programs_df.columns if c.strip().isdigit()]
    prog_numeric = programs_df[year_cols].apply(pd.to_numeric, errors='coerce').fillna(0)
    
    macro_features = pd.DataFrame()
    macro_features["Year"] = [int(y) for y in year_cols]
    macro_features["Global_Total_Events"] = prog_numeric.sum().values
    macro_features = macro_features.merge(
        athletes_df.groupby("Year")["NOC"].nunique().reset_index(name="Global_Nations_Count"),
        on="Year", how="left"
    )
    # 只要2024及以前
    macro_features = macro_features[macro_features["Year"] <= 2024]
    
    # --- B. 微观特征 ---
    # 1. 队伍规模 (多少人)
    squad_size = athletes_df.groupby(["Year", "NOC"])["Name"].nunique().reset_index(name="Squad_Size")
    # 2. 参赛项目数 (参加了多少个小项)
    noc_events = athletes_df.groupby(["Year", "NOC"])["Event"].nunique().reset_index(name="NOC_Events_Entered")
    # 3. 体育多样性 (参加了多少个大项)
    sport_diversity = athletes_df.groupby(["Year", "NOC"])["Sport"].nunique().reset_index(name="Sport_Diversity")
    
    # 合并微观特征
    micro_features = squad_size.merge(noc_events, on=["Year", "NOC"], how="left")
    micro_features = micro_features.merge(sport_diversity, on=["Year", "NOC"], how="left")
    
    # --- C. 合并奖牌数据 ---
    target_cols = ["Year", "NOC_Code", "Total"]
    features_with_medals = micro_features.merge(
        medals_df[target_cols], 
        left_on=["Year", "NOC"], 
        right_on=["Year", "NOC_Code"], 
        how="left"
    )
    features_with_medals["Total"] = features_with_medals["Total"].fillna(0).astype(int)
    features_with_medals = features_with_medals.drop(columns=['NOC_Code'])
    
    # --- D. 合并宏观特征并添加时序特征 ---
    basic_features_df = features_with_medals.merge(macro_features, on="Year", how="left")
    basic_features_df = basic_features_df.sort_values(["NOC", "Year"])
    
    # 历史累计表现
    basic_features_df["Historical_Total"] = basic_features_df.groupby("NOC", group_keys=False)["Total"].apply(
        lambda x: x.cumsum().shift(1)
    ).fillna(0)
    
    return basic_features_df

basic_features_df = build_basic_features(ath, med, prog)


>>> [3/4] 构建特征工程 (宏观+微观)...


In [29]:
# ============================================================
# 3. 构建特征 (宏观 + 微观)
# ============================================================
from typing import final


def build_features(ath, med, prog):
    # --- A. 宏观特征 ---
    year_cols = [c for c in prog.columns if c.strip().isdigit()]
    prog_numeric = prog[year_cols].apply(pd.to_numeric, errors='coerce').fillna(0)

    macro_stats = pd.DataFrame()
    macro_stats["Year"] = [int(y) for y in year_cols]
    macro_stats["Global_Total_Events"] = prog_numeric.sum().values
    macro_stats = macro_stats.merge(
        ath.groupby("Year")["NOC"].nunique().reset_index(name="Global_Nations_Count"),
        on="Year", how="left"
    )
    # 只要2024及以前
    macro_stats = macro_stats[macro_stats["Year"] <= 2024]

    # --- B. 微观特征 (【关键修改点】：加入了 Sport_Diversity) ---
    # 1. 队伍规模 (多少人)
    squad = ath.groupby(["Year", "NOC"])["Name"].nunique().reset_index(name="Squad_Size")
    # 2. 参赛项目数 (参加了多少个小项)
    noc_events = ath.groupby(["Year", "NOC"])["Event"].nunique().reset_index(name="NOC_Events_Entered")
    # 3. 体育多样性 (参加了多少个大项) -> 也就是 "Specialization Index" 的分母
    sport_div = ath.groupby(["Year", "NOC"])["Sport"].nunique().reset_index(name="Sport_Diversity")

    # 合并微观特征
    features = squad.merge(noc_events, on=["Year", "NOC"], how="left")
    features = features.merge(sport_div, on=["Year", "NOC"], how="left")

    # --- C. 合并奖牌 ---
    target_cols = ["Year", "NOC_Code", "Total"]
    master = features.merge(med[target_cols], left_on=["Year", "NOC"], right_on=["Year", "NOC_Code"], how="left")
    master["Total"] = master["Total"].fillna(0).astype(int)
    master = master.drop(columns=['NOC_Code'])

    # --- D. 关键时序特征 ---
    master = master.sort_values(["NOC", "Year"])

    # 2. 历史累计表现 (使用 group_keys=False 修复报错)
    master["Historical_Total"] = master.groupby("NOC", group_keys=False)["Total"].apply(
        lambda x: x.cumsum().shift(1)
    ).fillna(0)


    final_df = master.merge(macro_stats, on="Year", how="left")

    # --- 【新增核心逻辑 Start】 ---

    # 1. 计算宏观参与 RCA (RCA_Macro)
    # 分子：该国在当年的项目份额 (Share of Entries)
    yearly_total_entries = final_df.groupby("Year")["NOC_Events_Entered"].transform("sum")
    final_df["Share_of_Entries"] = final_df["NOC_Events_Entered"] / yearly_total_entries

    # 分母：期望份额 (Expected Share = 1 / Global Nations Count)
    final_df["Expected_Share"] = 1 / final_df["Global_Nations_Count"]

    # RCA 计算
    final_df["RCA_Macro"] = final_df["Share_of_Entries"] / final_df["Expected_Share"]
    # 填充可能产生的 inf (虽然不太可能，以防万一)
    final_df["RCA_Macro"] = final_df["RCA_Macro"].replace([np.inf, -np.inf], 0).fillna(0)

    # 2. 计算专注度指数 (Specialization Index)
    # 公式：Events_Entered / Sport_Diversity
    # 含义：平均每个大项报了多少个小项。数值高说明“兵力集中”。
    # 防止除以0
    final_df["Sport_Diversity"] = final_df["Sport_Diversity"].replace(0, 1)
    final_df["Specialization_Index"] = final_df["NOC_Events_Entered"] / final_df["Sport_Diversity"]

    # --- 【新增核心逻辑 End】 ---

    # 东道主处理
    hst["HostCountry"] = hst["Host"].apply(lambda x: x.split(",")[-1].strip())
    hst["HostNOC"] = hst["HostCountry"].map(name_to_code_map)
    hst_map = {
        "United Kingdom": "GBR", "Australia": "AUS", "Japan": "JPN",
        "China": "CHN", "Brazil": "BRA", "Greece": "GRE"
    }
    hst["HostNOC"] = hst["HostNOC"].fillna(hst["HostCountry"].map(hst_map))
    host_dict = hst.set_index("Year")["HostNOC"].to_dict()

    # 计算 Host 特征
    final_df["Is_Host"] = final_df.apply(lambda r: 1 if host_dict.get(r["Year"]) == r["NOC"] else 0, axis=1)

    # 曾经是否得过奖
    final_df["Has_Medal_before"] = (final_df["Historical_Total"] > 0).astype(int)


    return final_df

fe_df = build_features(ath, med, prog)
fe_df.head(100)

,Year,NOC,Squad_Size,NOC_Events_Entered,Sport_Diversity,Total,Historical_Total,Global_Total_Events,Global_Nations_Count,Share_of_Entries,Expected_Share,RCA_Macro,Specialization_Index,Is_Host,Has_Medal_before
0,1936,AFG,15,4,2,0,0.0,302.0,49,0.001942,0.020408,0.095146,2.000000,0,0
1,1948,AFG,22,2,2,0,0.0,312.0,59,0.000905,0.016949,0.053370,1.000000,0,0
2,1956,AFG,12,1,1,0,0.0,342.0,72,0.000461,0.013889,0.033164,1.000000,0,0
3,1960,AFG,12,13,2,0,0.0,340.0,84,0.003861,0.011905,0.324324,6.500000,0,0
4,1964,AFG,8,8,1,0,0.0,370.0,93,0.002476,0.010753,0.230269,8.000000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1924,ARG,77,39,11,6,0.0,292.0,44,0.024104,0.022727,1.060569,3.545455,0,0
96,1928,ARG,81,40,12,7,6.0,252.0,46,0.026110,0.021739,1.201044,3.333333,0,1
97,1932,ARG,32,29,6,0,13.0,268.0,37,0.033143,0.027027,1.226286,4.833333,0,1
98,1936,ARG,51,31,8,7,13.0,302.0,49,0.015049,0.020408,0.737379,3.875000,0,1


In [30]:
def lag_features(df, host_dict=None):
    """创建滞后特征，并为2028年创建预测数据（使用2024年的滞后特征）"""
    df = df.copy()
    # 确保按 NOC 和 Year 排序
    df = df.sort_values(["NOC", "Year"]).reset_index(drop=True)
    
    # 需要滞后的列
    need_lag_cols = [
        "Squad_Size",
        "NOC_Events_Entered",
        "Sport_Diversity",
        "Global_Total_Events",
        "Global_Nations_Count",
        "Share_of_Entries",
        "Expected_Share",
        "RCA_Macro",
        "Specialization_Index",
    ]
    
    # 分离训练数据（2024及以前）
    train_data = df[df["Year"] <= 2024].copy()
    
    # 为训练数据创建滞后特征
    res_df = pd.DataFrame()
    res_df["Year"] = train_data["Year"]
    res_df["NOC"] = train_data["NOC"]
    
    # 创建滞后特征：使用上一年的数据
    for col in need_lag_cols:
        res_df[f"Lagged_{col}"] = train_data.groupby("NOC")[col].shift(1)
    
    # 创建奖牌的滞后特征
    res_df["Lagged_Total_Medals"] = train_data.groupby("NOC")["Total"].shift(1)
    res_df["Lagged_2_Total_Medals"] = train_data.groupby("NOC")["Total"].shift(2)
    res_df["Lagged_3_Total_Medals"] = train_data.groupby("NOC")["Total"].shift(3)
    
    # 复制非滞后特征
    res_df["Is_Host"] = train_data["Is_Host"]
    res_df["Has_Medal_before"] = train_data["Has_Medal_before"]
    res_df["Total"] = train_data["Total"]  # 保留目标变量用于训练
    
    # ============================================================
    # 为2028年创建预测数据（使用2024年的特征作为滞后特征）
    # ============================================================
    print(">>> 为2028年创建预测数据，使用2024年的特征作为滞后特征...")
    
    # 获取2024年的所有NOC数据
    df_2024 = train_data[train_data["Year"] == 2024].copy()
    
    if len(df_2024) == 0:
        print(">>> 警告：未找到2024年数据，无法创建2028年预测数据")
        df_2028 = pd.DataFrame()
        return res_df, df_2028
    
    # 为每个在2024年有数据的NOC创建2028年记录
    df_2028_lagged = pd.DataFrame()
    df_2028_lagged["Year"] = [2028] * len(df_2024)
    df_2028_lagged["NOC"] = df_2024["NOC"].values
    
    # 使用2024年的特征值作为滞后特征
    for col in need_lag_cols:
        df_2028_lagged[f"Lagged_{col}"] = df_2024[col].values
    
    # 创建奖牌的滞后特征（使用历史数据）
    lagged_medals_1 = []
    lagged_medals_2 = []
    lagged_medals_3 = []
    
    for noc in df_2024["NOC"]:
        noc_history = train_data[train_data["NOC"] == noc].sort_values("Year", ascending=False)
        if len(noc_history) >= 1:
            lagged_medals_1.append(noc_history["Total"].iloc[0])  # 2024年
        else:
            lagged_medals_1.append(0)
        
        if len(noc_history) >= 2:
            lagged_medals_2.append(noc_history["Total"].iloc[1])  # 2020年
        else:
            lagged_medals_2.append(0)
        
        if len(noc_history) >= 3:
            lagged_medals_3.append(noc_history["Total"].iloc[2])  # 2016年
        else:
            lagged_medals_3.append(0)
    
    df_2028_lagged["Lagged_Total_Medals"] = lagged_medals_1
    df_2028_lagged["Lagged_2_Total_Medals"] = lagged_medals_2
    df_2028_lagged["Lagged_3_Total_Medals"] = lagged_medals_3
    
    # 处理Has_Medal_before特征（基于2024年及之前的历史累计）
    df_2028_lagged["Has_Medal_before"] = df_2024["Has_Medal_before"].values
    
    # 处理Is_Host特征（2028年是洛杉矶奥运会，USA是东道主）
    if host_dict is not None:
        df_2028_lagged["Is_Host"] = df_2028_lagged["NOC"].apply(
            lambda noc: 1 if host_dict.get(2028) == noc else 0
        )
    else:
        # 如果没有提供host_dict，默认USA是2028年东道主
        df_2028_lagged["Is_Host"] = (df_2028_lagged["NOC"] == "USA").astype(int)
    
    return res_df, df_2028_lagged

train_df, test_df = lag_features(fe_df)

# ============================================================
# 合并金银铜数据到train_df
# ============================================================
print("\n>>> 合并金银铜数据到训练数据...")

# 从med数据中提取金银铜列，使用NOC_Code和Year对齐
medal_cols = med[["NOC_Code", "Year", "Gold", "Silver", "Bronze"]].copy()
medal_cols = medal_cols.rename(columns={"NOC_Code": "NOC"})

# 合并到train_df，对齐NOC和Year
train_df = train_df.merge(
    medal_cols,
    on=["NOC", "Year"],
    how="left"
)

# 填充缺失值（某些NOC在某个年份可能没有奖牌数据）
train_df["Gold"] = train_df["Gold"].fillna(0).astype(int)
train_df["Silver"] = train_df["Silver"].fillna(0).astype(int)
train_df["Bronze"] = train_df["Bronze"].fillna(0).astype(int)


test_df.head()

train_df.to_csv("./data/train_df.csv", index=False)


>>> 为2028年创建预测数据，使用2024年的特征作为滞后特征...

>>> 合并金银铜数据到训练数据...


In [31]:
# ============================================================
# 使用XGBoost预测2028年金银铜牌数量
# ============================================================
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

print(">>> 开始训练XGBoost模型预测金银铜牌...")

# 准备训练数据：删除缺失值过多的行
train_df_clean = train_df.dropna().copy()
print(f">>> 训练数据行数: {len(train_df_clean)} (删除缺失值后)")

# 定义特征列（排除目标变量和标识列）
feature_cols = [
    'Lagged_Squad_Size',
    'Lagged_NOC_Events_Entered',
    'Lagged_Sport_Diversity',
    'Lagged_Global_Total_Events',
    'Lagged_Global_Nations_Count',
    'Lagged_Share_of_Entries',
    'Lagged_Expected_Share',
    'Lagged_RCA_Macro',
    'Lagged_Specialization_Index',
    'Lagged_Total_Medals',
    'Lagged_2_Total_Medals',
    'Lagged_3_Total_Medals',
    'Is_Host',
    'Has_Medal_before'
]

# 检查特征列是否存在
available_features = [col for col in feature_cols if col in train_df_clean.columns]
print(f">>> 可用特征数: {len(available_features)}")

# 准备训练数据
X_train = train_df_clean[available_features].copy()
y_gold = train_df_clean['Gold'].copy()
y_silver = train_df_clean['Silver'].copy()
y_bronze = train_df_clean['Bronze'].copy()

# 准备测试数据（2028年）
# 检查test_df是否有缺失值，如果有则填充
test_df_clean = test_df.copy()
for col in available_features:
    if col in test_df_clean.columns:
        if test_df_clean[col].isna().any():
            # 使用中位数填充
            median_val = train_df_clean[col].median()
            test_df_clean[col] = test_df_clean[col].fillna(median_val)
            print(f">>> 填充 {col} 的缺失值，使用中位数: {median_val}")

X_test = test_df_clean[available_features].copy()

print(f"\n>>> 训练集大小: {len(X_train)}")
print(f">>> 测试集大小: {len(X_test)}")

# 训练三个模型分别预测金银铜
models = {}
predictions = {}

for target_name, y_target in [('Gold', y_gold), ('Silver', y_silver), ('Bronze', y_bronze)]:
    print(f"\n>>> 训练 {target_name} 预测模型...")
    
    # 分割训练集和验证集
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_train, y_target, test_size=0.2, random_state=42
    )
    
    # 创建XGBoost模型
    model = xgb.XGBRegressor(
        n_estimators=200,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        objective='reg:squarederror'
    )
    
    # 训练模型
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        verbose=False
    )
    
    # 验证集预测
    y_pred_val = model.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
    
    print(f">>> {target_name} 模型 - MAE: {mae:.2f}, RMSE: {rmse:.2f}")
    
    # 保存模型
    models[target_name] = model
    
    # 对2028年数据进行预测
    pred_2028 = model.predict(X_test)
    predictions[target_name] = pred_2028

# 创建预测结果DataFrame
pred_df_2028 = test_df_clean[['Year', 'NOC']].copy()
pred_df_2028['Gold'] = np.round(predictions['Gold']).astype(int)
pred_df_2028['Silver'] = np.round(predictions['Silver']).astype(int)
pred_df_2028['Bronze'] = np.round(predictions['Bronze']).astype(int)
pred_df_2028['Total'] = pred_df_2028['Gold'] + pred_df_2028['Silver'] + pred_df_2028['Bronze']

# 确保预测值非负
pred_df_2028['Gold'] = pred_df_2028['Gold'].clip(lower=0)
pred_df_2028['Silver'] = pred_df_2028['Silver'].clip(lower=0)
pred_df_2028['Bronze'] = pred_df_2028['Bronze'].clip(lower=0)
pred_df_2028['Total'] = pred_df_2028['Total'].clip(lower=0)

print("\n>>> 2028年预测结果:")
print(f">>> 预测国家数: {len(pred_df_2028)}")
print(f">>> 预计获得奖牌的国家数: {(pred_df_2028['Total'] > 0).sum()}")
print(f"\n>>> 2028年预测奖牌总数:")
print(f">>> 金牌: {pred_df_2028['Gold'].sum()}")
print(f">>> 银牌: {pred_df_2028['Silver'].sum()}")
print(f">>> 铜牌: {pred_df_2028['Bronze'].sum()}")
print(f">>> 总计: {pred_df_2028['Total'].sum()}")

# 显示前20名预测结果
print("\n>>> 2028年预测奖牌数前20名:")
top_20 = pred_df_2028.nlargest(20, 'Total')[['NOC', 'Gold', 'Silver', 'Bronze', 'Total']]
print(top_20.to_string(index=False))

# 保存预测结果
pred_df_2028 = pred_df_2028.sort_values(by='Total', ascending=False)
pred_df_2028.to_csv("outputs/lstm_pred_2028.csv", index=False)
print(f"\n>>> 预测结果已保存到: outputs/lstm_pred_2028.csv")

pred_df_2028.head(20)

>>> 开始训练XGBoost模型预测金银铜牌...
>>> 训练数据行数: 2519 (删除缺失值后)
>>> 可用特征数: 14

>>> 训练集大小: 2519
>>> 测试集大小: 204

>>> 训练 Gold 预测模型...
>>> Gold 模型 - MAE: 1.12, RMSE: 2.95

>>> 训练 Silver 预测模型...
>>> Silver 模型 - MAE: 0.98, RMSE: 2.22

>>> 训练 Bronze 预测模型...
>>> Bronze 模型 - MAE: 1.14, RMSE: 2.46

>>> 2028年预测结果:
>>> 预测国家数: 204
>>> 预计获得奖牌的国家数: 89

>>> 2028年预测奖牌总数:
>>> 金牌: 314
>>> 银牌: 324
>>> 铜牌: 354
>>> 总计: 992

>>> 2028年预测奖牌数前20名:
NOC  Gold  Silver  Bronze  Total
USA    45      45      35    125
CHN    29      20      26     75
GBR    18      24      27     69
AUS    18      16      14     48
FRA    17      13      18     48
JPN    15      17      14     46
ITA    15      14      16     45
GER     9      12      14     35
NED    11      12      10     33
CAN     8       7      14     29
ESP     7       8       8     23
HUN     7       7       8     22
BRA     7       7       7     21
KOR     5       6       9     20
NZL     6       5       7     18
POL     2       5       5     12
UKR     3       4       

,Year,NOC,Gold,Silver,Bronze,Total
195,2028,USA,45,45,35,125
37,2028,CHN,29,20,26,75
67,2028,GBR,18,24,27,69
10,2028,AUS,18,16,14,48
63,2028,FRA,17,13,18,48
95,2028,JPN,15,17,14,46
91,2028,ITA,15,14,16,45
71,2028,GER,9,12,14,35
134,2028,NED,11,12,10,33
32,2028,CAN,8,7,14,29


In [ ]:
# ============================================================
# 模型验证：使用2024年以前的数据预测2024年成绩
# ============================================================
print("=" * 60)
print(">>> 开始模型验证：预测2024年成绩")
print("=" * 60)

# 分离训练数据（2024年以前）和验证数据（2024年）
train_df_before_2024 = train_df[train_df['Year'] < 2024].copy()
train_df_2024 = train_df[train_df['Year'] == 2024].copy()

print(f"\n>>> 训练数据（2024年以前）: {len(train_df_before_2024)} 条")
print(f">>> 验证数据（2024年）: {len(train_df_2024)} 条")

# 准备训练数据：删除缺失值
train_df_clean = train_df_before_2024.dropna().copy()
print(f">>> 清理后的训练数据: {len(train_df_clean)} 条")

# 准备验证数据：填充缺失值
val_df_clean = train_df_2024.copy()
for col in available_features:
    if col in val_df_clean.columns:
        if val_df_clean[col].isna().any():
            median_val = train_df_clean[col].median()
            val_df_clean[col] = val_df_clean[col].fillna(median_val)

# 准备特征和目标变量
X_train_val = train_df_clean[available_features].copy()
X_val_2024 = val_df_clean[available_features].copy()

y_gold_train = train_df_clean['Gold'].copy()
y_silver_train = train_df_clean['Silver'].copy()
y_bronze_train = train_df_clean['Bronze'].copy()

y_gold_actual = val_df_clean['Gold'].copy()
y_silver_actual = val_df_clean['Silver'].copy()
y_bronze_actual = val_df_clean['Bronze'].copy()

print(f"\n>>> 训练集大小: {len(X_train_val)}")
print(f">>> 验证集大小: {len(X_val_2024)}")

# 训练模型并预测2024年
models_val = {}
predictions_val = {}
metrics_val = {}

for target_name, y_train, y_actual in [
    ('Gold', y_gold_train, y_gold_actual),
    ('Silver', y_silver_train, y_silver_actual),
    ('Bronze', y_bronze_train, y_bronze_actual)
]:
    print(f"\n>>> 训练 {target_name} 预测模型（使用2024年以前数据）...")
    
    # 创建XGBoost模型
    model = xgb.XGBRegressor(
        n_estimators=150,
        max_depth=5,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        objective='reg:squarederror'
    )
    
    # 训练模型
    model.fit(
        X_train_val, y_train,
        verbose=False
    )
    
    # 预测2024年
    y_pred_2024 = model.predict(X_val_2024)
    y_pred_2024 = np.round(y_pred_2024).astype(int)
    y_pred_2024 = np.clip(y_pred_2024, 0, None)  # 确保非负
    
    # 计算评估指标
    mae = mean_absolute_error(y_actual, y_pred_2024)
    rmse = np.sqrt(mean_squared_error(y_actual, y_pred_2024))
    
    # 计算准确率（完全匹配的比例）
    exact_match = (y_pred_2024 == y_actual).sum() / len(y_actual)
    
    # 计算误差在±1范围内的比例
    within_1 = (np.abs(y_pred_2024 - y_actual) <= 1).sum() / len(y_actual)
    
    # 计算误差在±2范围内的比例
    within_2 = (np.abs(y_pred_2024 - y_actual) <= 2).sum() / len(y_actual)
    
    print(f">>> {target_name} 模型评估指标:")
    print(f">>>   MAE: {mae:.2f}")
    print(f">>>   RMSE: {rmse:.2f}")
    print(f">>>   完全匹配率: {exact_match*100:.2f}%")
    print(f">>>   误差≤1的比例: {within_1*100:.2f}%")
    print(f">>>   误差≤2的比例: {within_2*100:.2f}%")
    
    models_val[target_name] = model
    predictions_val[target_name] = y_pred_2024
    metrics_val[target_name] = {
        'MAE': mae,
        'RMSE': rmse,
        'Exact_Match': exact_match,
        'Within_1': within_1,
        'Within_2': within_2
    }

# 创建对比结果DataFrame
comparison_df = val_df_clean[['Year', 'NOC']].copy()
comparison_df['Gold_Actual'] = y_gold_actual.values
comparison_df['Gold_Pred'] = predictions_val['Gold']
comparison_df['Gold_Error'] = comparison_df['Gold_Pred'] - comparison_df['Gold_Actual']

comparison_df['Silver_Actual'] = y_silver_actual.values
comparison_df['Silver_Pred'] = predictions_val['Silver']
comparison_df['Silver_Error'] = comparison_df['Silver_Pred'] - comparison_df['Silver_Actual']

comparison_df['Bronze_Actual'] = y_bronze_actual.values
comparison_df['Bronze_Pred'] = predictions_val['Bronze']
comparison_df['Bronze_Error'] = comparison_df['Bronze_Pred'] - comparison_df['Bronze_Actual']

comparison_df['Total_Actual'] = comparison_df['Gold_Actual'] + comparison_df['Silver_Actual'] + comparison_df['Bronze_Actual']
comparison_df['Total_Pred'] = comparison_df['Gold_Pred'] + comparison_df['Silver_Pred'] + comparison_df['Bronze_Pred']
comparison_df['Total_Error'] = comparison_df['Total_Pred'] - comparison_df['Total_Actual']

# 计算总奖牌数的评估指标
total_mae = mean_absolute_error(comparison_df['Total_Actual'], comparison_df['Total_Pred'])
total_rmse = np.sqrt(mean_squared_error(comparison_df['Total_Actual'], comparison_df['Total_Pred']))
total_exact_match = (comparison_df['Total_Pred'] == comparison_df['Total_Actual']).sum() / len(comparison_df)
total_within_1 = (np.abs(comparison_df['Total_Error']) <= 1).sum() / len(comparison_df)
total_within_2 = (np.abs(comparison_df['Total_Error']) <= 2).sum() / len(comparison_df)

print("\n" + "=" * 60)
print(">>> 总奖牌数预测评估:")
print(f">>>   MAE: {total_mae:.2f}")
print(f">>>   RMSE: {total_rmse:.2f}")
print(f">>>   完全匹配率: {total_exact_match*100:.2f}%")
print(f">>>   误差≤1的比例: {total_within_1*100:.2f}%")
print(f">>>   误差≤2的比例: {total_within_2*100:.2f}%")
print("=" * 60)

# 显示预测最准确的前10名
print("\n>>> 预测最准确的前10名（按总奖牌数误差排序）:")
top_accurate = comparison_df.nsmallest(10, 'Total_Error', keep='all')[['NOC', 'Total_Actual', 'Total_Pred', 'Total_Error']]
print(top_accurate.to_string(index=False))

# 显示预测误差最大的前10名
print("\n>>> 预测误差最大的前10名:")
top_error = comparison_df.nlargest(10, 'Total_Error', keep='all')[['NOC', 'Total_Actual', 'Total_Pred', 'Total_Error']]
print(top_error.to_string(index=False))

# 显示实际奖牌数前20名的预测对比
print("\n>>> 实际奖牌数前20名的预测对比:")
top_20_actual = comparison_df.nlargest(20, 'Total_Actual')[['NOC', 'Gold_Actual', 'Gold_Pred', 'Silver_Actual', 'Silver_Pred', 'Bronze_Actual', 'Bronze_Pred', 'Total_Actual', 'Total_Pred', 'Total_Error']]
print(top_20_actual.to_string(index=False))

# 保存对比结果
comparison_df = comparison_df.sort_values(by='Total_Actual', ascending=False)
comparison_df.to_csv("outputs/pred_vs_actual_2024.csv", index=False)
print(f"\n>>> 对比结果已保存到: outputs/pred_vs_actual_2024.csv")

comparison_df.head(20)

>>> 开始模型验证：预测2024年成绩

>>> 训练数据（2024年以前）: 2976 条
>>> 验证数据（2024年）: 204 条
>>> 清理后的训练数据: 2318 条

>>> 训练集大小: 2318
>>> 验证集大小: 204

>>> 训练 Gold 预测模型（使用2024年以前数据）...
>>> Gold 模型评估指标:
>>>   MAE: 0.68
>>>   RMSE: 1.74
>>>   完全匹配率: 67.16%
>>>   误差≤1的比例: 87.75%
>>>   误差≤2的比例: 94.12%

>>> 训练 Silver 预测模型（使用2024年以前数据）...
>>> Silver 模型评估指标:
>>>   MAE: 0.66
>>>   RMSE: 1.60
>>>   完全匹配率: 65.20%
>>>   误差≤1的比例: 87.75%
>>>   误差≤2的比例: 93.63%

>>> 训练 Bronze 预测模型（使用2024年以前数据）...
>>> Bronze 模型评估指标:
>>>   MAE: 0.72
>>>   RMSE: 1.57
>>>   完全匹配率: 62.75%
>>>   误差≤1的比例: 84.31%
>>>   误差≤2的比例: 92.65%

>>> 总奖牌数预测评估:
>>>   MAE: 1.30
>>>   RMSE: 3.24
>>>   完全匹配率: 59.31%
>>>   误差≤1的比例: 74.02%
>>>   误差≤2的比例: 82.35%

>>> 预测最准确的前10名（按总奖牌数误差排序）:
NOC  Total_Actual  Total_Pred  Total_Error
USA           126         101          -25
CHN            91          72          -19
FRA            64          47          -17
AUS            53          40          -13
NED            34          24          -10
KOR            32         

,Year,NOC,Gold_Actual,Gold_Pred,Gold_Error,Silver_Actual,Silver_Pred,Silver_Error,Bronze_Actual,Bronze_Pred,Bronze_Error,Total_Actual,Total_Pred,Total_Error
3055,2024,USA,40,35,-5,44,34,-10,42,32,-10,126,101,-25
599,2024,CHN,40,27,-13,27,21,-6,24,24,0,91,72,-19
1072,2024,GBR,14,25,11,22,20,-2,29,21,-8,65,66,1
1007,2024,FRA,16,20,4,26,14,-12,22,13,-9,64,47,-17
173,2024,AUS,18,14,-4,19,12,-7,16,14,-2,53,40,-13
1569,2024,JPN,20,19,-1,12,16,4,13,15,2,45,50,5
1503,2024,ITA,12,15,3,13,12,-1,15,12,-3,40,39,-1
2097,2024,NED,15,8,-7,7,6,-1,12,10,-2,34,24,-10
1125,2024,GER,12,13,1,13,10,-3,8,13,5,33,36,3
1626,2024,KOR,13,8,-5,9,7,-2,10,8,-2,32,23,-9
